In [ ]:
import os
import numpy as np
import keras.layers
import matplotlib.pyplot as plt
#import tensorflow as tf
from sklearn.metrics import mean_squared_error
import h5py
import scipy
import skimage
import random
import csv
import cv2
import index_tracker as track  
from keras.callbacks import LearningRateScheduler
import time

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# set seed
from numpy.random import seed
seed(1234)
from tensorflow import set_random_seed
set_random_seed(1234)


from MaskRCNN import maskrcnn_model as modellib, data_generator as dg, config as conf, visualize, utils, metrics
from mrcnn import test_mrcnn

**1. Set test hyperparameters**

In [ ]:
# config
config = conf.Config()
config.NAME = 'lung_tumor'
config.BACKBONE = "resnet50"
config.BATCH_SIZE =  1
config.IMAGES_PER_GPU = 1
TRAIN_BN = True
config.IMAGE_HEIGHT = 384
config.IMAGE_WIDTH = 384
config.IMAGE_SHAPE = [384, 384, 1]
config.MASK = False
config.FPN_CLASSIF_FC_LAYERS_SIZE = 256
config.DETECTION_MIN_CONFIDENCE = 0.1

data_path = ''   
model = 'mrcnn'
data_mode = 'standard'

**2. Load DRR test data**

In [ ]:
# load data
patient_name = 'patient1'
model_path = data_path + patient_name + '/models/' + model + '/' + data_mode
drr_data_path = model_path + '/dataset/'    
LOG_DIR = model_path + '/logs/'

test_imgs = load_data(drr_data_path + 'test_imgs.hdf5', 'test_imgs')
test_labs = load_data(drr_data_path + 'test_labs.hdf5', 'test_labs')

**3. Preprocess DRR test data**

In [ ]:
# get the center 384x384 region
test_imgs = test_imgs[..., 64:-64]
test_labs = test_labs[..., 64:-64]
for i in range(test_labs.shape[0]):
    for j in range(test_labs.shape[1]):
        test_labs[i, j] = fit_ellipse(test_labs[i, j]) 
test_imgs = (test_imgs+1)/2.

# TRAIN MODEL
s = test_imgs.shape
test_imgs = np.reshape(test_imgs, (-1, s[2], s[3]))
test_labs = np.reshape(test_labs, (-1, s[2], s[3]))

**4. Load trained model**

In [ ]:
# load model
test_model = modellib.MaskRCNN(mode="inference", model_dir=LOG_DIR,
                          config=config)    
weights_path = test_model.find_last()
# Load weights
print("Loading weights ", weights_path)
test_model.load_weights(weights_path, by_name=True)    

**5. Test model**

In [ ]:
gt, pred, nr_pred = test_model.test(test_imgs_mrcnn, test_labs_mrcnn, verbose=0)    
# get groud truth data
gt_bboxs = gt['gt_bboxs']
gt_class_ids =  gt['gt_class_ids']
gt_masks =  gt['gt_masks']
# get predicted data
pred_bboxs =  pred['pred_bboxs'][:, np.newaxis, ...]
pred_class_ids =  pred['pred_class_ids'][:, np.newaxis, ...]
pred_scores =  pred['pred_scores'][:, np.newaxis, ...]

**6. Compute metrics**

In [ ]:
miou = metrics.mIOU(gt_bboxs, pred_bboxs)
accuracy = metrics.accuracy(gt_bboxs, pred_bboxs, 
                            iou_threshold = 0.7)
cor_z, cor_x = metrics.concordance_correlation_coefficient(gt_bboxs, pred_bboxs)
z_mad, z_std, x_mad, x_std = metrics.mad(gt_bboxs, pred_bboxs)    
z_mad, z_std, x_mad, x_std = z_mad/3, z_std/3, x_mad/3, x_std/3 

**7. Load X-Rays**

In [ ]:
data_path = ''
patient_name = 'patient2'
xray_data_path = data_path + patient_name + '/X_rays/'

xrays = load_data(xray_data_path + 'patient2_x_rays_1.h5', 'img')
angles = load_data(xray_data_path + 'patient2_x_rays_1.h5', 'rot')
angles = angles-270

**8. Preprocess X-Rays**

In [ ]:
xrays = utils.log_transform(xrays)

shape = np.array(xrays.shape[1:])
new_shape = [384, 384]
xrays = xrays[:, shape[0]//2-new_shape[0]//2:shape[0]//2+new_shape[0]//2, 
                shape[1]//2-new_shape[1]//2:shape[1]//2+new_shape[1]//2] 

xrays = (xrays-np.min(xrays))/(np.max(xrays)-np.min(xrays))
xrays = utils.gamma_transform(xrays, 5)

xrays_mrcnn = xrays[..., 64:-64, np.newaxis]

**9. Test on X-Rays**

In [ ]:
nr_pred = 0
pred_bboxs, pred_class_ids, pred_masks, pred_scores = [], [], [], [] 
for i in range(xrays_mrcnn.shape[0]):  
    result = test_model.detect([xrays_mrcnn[i]], 0)
    if len(result[0]['scores']) > 0:
        max_idx = np.argmax(result[0]['scores'])
        pred_bboxs.append(result[0]['rois'][max_idx])
        pred_class_ids.append(result[0]['class_ids'][max_idx])
        pred_scores.append(result[0]['scores'][max_idx])
        nr_pred += 1
        print(pred_bboxs[-1])
    else:
        pred_bboxs.append(pred_bboxs[-1])
        pred_class_ids.append(pred_class_ids[-1])
        pred_scores.append(0)
        print(pred_bboxs[-1])


nr_pred = nr_pred/xrays_mrcnn.shape[0] 
pred_bboxs = np.stack(pred_bboxs, axis = 0)
pred_class_ids = np.stack(pred_class_ids, axis = 0)
pred_scores = np.stack(pred_scores, axis = 0)